### Importing Libraries

In [1]:
import ast
import pymongo
import json
import pandas as pd
import numpy as np
from pprint import pprint
pd.set_option('display.max_columns', None)

In [1]:
# client = pymongo.MongoClient("link_from_your_mongodb_atlas")

In [3]:
db = client['sample_airbnb']
collection = db['listingsAndReviews']

In [4]:
data = list(collection.find())

In [5]:
df = pd.DataFrame(data)

In [6]:
df.shape

(5555, 42)

In [7]:
df.columns

Index(['_id', 'listing_url', 'name', 'summary', 'space', 'description',
       'neighborhood_overview', 'notes', 'transit', 'access', 'interaction',
       'house_rules', 'property_type', 'room_type', 'bed_type',
       'minimum_nights', 'maximum_nights', 'cancellation_policy',
       'last_scraped', 'calendar_last_scraped', 'first_review', 'last_review',
       'accommodates', 'bedrooms', 'beds', 'number_of_reviews', 'bathrooms',
       'amenities', 'price', 'security_deposit', 'cleaning_fee',
       'extra_people', 'guests_included', 'images', 'host', 'address',
       'availability', 'review_scores', 'reviews', 'weekly_price',
       'monthly_price', 'reviews_per_month'],
      dtype='object')

In [8]:
df.drop(['summary','space','description','neighborhood_overview','notes','transit','access',
          'interaction','house_rules','weekly_price','monthly_price', 'reviews_per_month','last_scraped',
               'calendar_last_scraped','last_review','reviews','first_review','security_deposit'],axis = 1, inplace = True)

In [9]:
df.shape

(5555, 24)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5555 entries, 0 to 5554
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   _id                  5555 non-null   object 
 1   listing_url          5555 non-null   object 
 2   name                 5555 non-null   object 
 3   property_type        5555 non-null   object 
 4   room_type            5555 non-null   object 
 5   bed_type             5555 non-null   object 
 6   minimum_nights       5555 non-null   object 
 7   maximum_nights       5555 non-null   object 
 8   cancellation_policy  5555 non-null   object 
 9   accommodates         5555 non-null   int64  
 10  bedrooms             5550 non-null   float64
 11  beds                 5542 non-null   float64
 12  number_of_reviews    5555 non-null   int64  
 13  bathrooms            5545 non-null   object 
 14  amenities            5555 non-null   object 
 15  price                5555 non-null   o

In [11]:
df.isnull().sum()

_id                       0
listing_url               0
name                      0
property_type             0
room_type                 0
bed_type                  0
minimum_nights            0
maximum_nights            0
cancellation_policy       0
accommodates              0
bedrooms                  5
beds                     13
number_of_reviews         0
bathrooms                10
amenities                 0
price                     0
cleaning_fee           1531
extra_people              0
guests_included           0
images                    0
host                      0
address                   0
availability              0
review_scores             0
dtype: int64

In [12]:
df.dropna(subset=['beds','bedrooms'], inplace=True)

### Host column

In [13]:
df['host'] = df['host'].astype(str)
df['host'] = df['host'].apply(ast.literal_eval)

In [14]:
host_columns_to_extract = ['host_id', 'host_url', 'host_name', 'host_location',
                       'host_response_time', 'host_thumbnail_url', 'host_picture_url',
                       'host_neighbourhood', 'host_response_rate', 'host_is_superhost',
                       'host_has_profile_pic', 'host_identity_verified',
                       'host_listings_count', 'host_total_listings_count',
                       'host_verifications']

In [15]:
for column in host_columns_to_extract:
    df[column] = df['host'].apply(lambda x: x.get(column))

In [16]:
df = df.drop('host',axis = 1)

In [17]:
mean_value = df['host_response_rate'].mean()
df['host_response_rate'].fillna(mean_value, inplace=True)

In [18]:
df['host_response_time'].unique()

array(['within an hour', None, 'within a few hours', 'within a day',
       'a few days or more'], dtype=object)

In [19]:
df['host_response_time'].fillna('Not Specified', inplace = True)

### Address Column

In [20]:
df['address'] = df['address'].astype(str)
df['address'] = df['address'].apply(ast.literal_eval)

In [21]:
address_columns_to_extract = ['street','suburb','government_area','market','country','country_code']

In [22]:
for column in address_columns_to_extract:
    df[column] = df['address'].apply(lambda x: x.get(column))

In [23]:
df['address'][0]['location']

{'type': 'Point',
 'coordinates': [-8.61308, 41.1413],
 'is_location_exact': False}

In [24]:
df['location_type'] = df['address'].apply(lambda x: x['location'].get('type'))
df['latitude'] = df['address'].apply(lambda x: x['location']['coordinates'][1])
df['longitude'] = df['address'].apply(lambda x: x['location']['coordinates'][0])
df['is_location_exact'] = df['address'].apply(lambda x: x['location'].get('is_location_exact'))

In [25]:
df.drop('address',axis = 1, inplace = True)

### Image Column

In [26]:
df['images'] = df['images'].astype(str)
df['images'] = df['images'].apply(ast.literal_eval)

In [27]:
df['images'].to_dict()[0].get('picture_url')

'https://a0.muscache.com/im/pictures/e83e702f-ef49-40fb-8fa0-6512d7e26e9b.jpg?aki_policy=large'

In [28]:
df['images'] = df['images'].apply(lambda x: x.get('picture_url'))

### Availability Column

In [29]:
df['availability'] = df['availability'].astype(str)
df['availability'] = df['availability'].apply(ast.literal_eval)

In [30]:
df.availability.to_dict()[0]

{'availability_30': 28,
 'availability_60': 47,
 'availability_90': 74,
 'availability_365': 239}

In [31]:
df['availability_30'] = df['availability'].apply(lambda x: x.get('availability_30'))
df['availability_60'] = df['availability'].apply(lambda x: x.get('availability_60'))
df['availability_90'] = df['availability'].apply(lambda x: x.get('availability_90'))
df['availability_365'] = df['availability'].apply(lambda x: x.get('availability_365'))

In [32]:
df.drop('availability',axis = 1, inplace = True)

In [33]:
df['review_scores'] = df['review_scores'].astype(str)
df['review_scores'] = df['review_scores'].apply(ast.literal_eval)
df['review_scores'] = df['review_scores'].apply(lambda x: x.get('review_scores_rating'))

### Data Conversion

In [34]:
df.isnull().sum()

_id                             0
listing_url                     0
name                            0
property_type                   0
room_type                       0
bed_type                        0
minimum_nights                  0
maximum_nights                  0
cancellation_policy             0
accommodates                    0
bedrooms                        0
beds                            0
number_of_reviews               0
bathrooms                       7
amenities                       0
price                           0
cleaning_fee                 1518
extra_people                    0
guests_included                 0
images                          0
review_scores                1462
host_id                         0
host_url                        0
host_name                       0
host_location                   0
host_response_time              0
host_thumbnail_url              0
host_picture_url                0
host_neighbourhood              0
host_response_

In [35]:
df.cleaning_fee.fillna('Not Specified', inplace = True)
df.review_scores.fillna(0,inplace = True)
df.bathrooms.fillna(1,inplace = True)

In [36]:
df.isnull().sum()

_id                          0
listing_url                  0
name                         0
property_type                0
room_type                    0
bed_type                     0
minimum_nights               0
maximum_nights               0
cancellation_policy          0
accommodates                 0
bedrooms                     0
beds                         0
number_of_reviews            0
bathrooms                    0
amenities                    0
price                        0
cleaning_fee                 0
extra_people                 0
guests_included              0
images                       0
review_scores                0
host_id                      0
host_url                     0
host_name                    0
host_location                0
host_response_time           0
host_thumbnail_url           0
host_picture_url             0
host_neighbourhood           0
host_response_rate           0
host_is_superhost            0
host_has_profile_pic         0
host_ide

In [37]:
for column in df.columns:
    if df[column].dtype == bool:
        df[column] = df[column].map({True: 1, False: 0})

In [38]:
columns_to_convert = ['minimum_nights', 'maximum_nights', 'bedrooms',
                      'beds' ,'review_scores']

df[columns_to_convert] = df[columns_to_convert].astype(int)

In [39]:
columns_to_convert = ['price','extra_people','guests_included']

df[columns_to_convert] = df[columns_to_convert].astype(str).astype(float).astype(int)

In [40]:
df.dtypes

_id                           object
listing_url                   object
name                          object
property_type                 object
room_type                     object
bed_type                      object
minimum_nights                 int32
maximum_nights                 int32
cancellation_policy           object
accommodates                   int64
bedrooms                       int32
beds                           int32
number_of_reviews              int64
bathrooms                     object
amenities                     object
price                          int32
cleaning_fee                  object
extra_people                   int32
guests_included                int32
images                        object
review_scores                  int32
host_id                       object
host_url                      object
host_name                     object
host_location                 object
host_response_time            object
host_thumbnail_url            object
h

In [41]:
df.to_csv('airbnb.csv', index = False)

In [43]:
df.head()

,_id,listing_url,name,property_type,room_type,bed_type,minimum_nights,maximum_nights,cancellation_policy,accommodates,bedrooms,beds,number_of_reviews,bathrooms,amenities,price,cleaning_fee,extra_people,guests_included,images,review_scores,host_id,host_url,host_name,host_location,host_response_time,host_thumbnail_url,host_picture_url,host_neighbourhood,host_response_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,host_listings_count,host_total_listings_count,host_verifications,street,suburb,government_area,market,country,country_code,location_type,latitude,longitude,is_location_exact,availability_30,availability_60,availability_90,availability_365
0,10006546,https://www.airbnb.com/rooms/10006546,Ribeira Charming Duplex,House,Entire home/apt,Real Bed,2,30,moderate,8,3,5,51,1.0,"[TV, Cable TV, Wifi, Kitchen, Paid parking off...",80,35.00,15,6,https://a0.muscache.com/im/pictures/e83e702f-e...,89,51399391,https://www.airbnb.com/users/show/51399391,Ana&Gonçalo,"Porto, Porto District, Portugal",within an hour,https://a0.muscache.com/im/pictures/fab79f25-2...,https://a0.muscache.com/im/pictures/fab79f25-2...,,100.000000,0,1,1,3,3,"[email, phone, reviews, jumio, offline_governm...","Porto, Porto, Portugal",,"Cedofeita, Ildefonso, Sé, Miragaia, Nicolau, V...",Porto,Portugal,PT,Point,41.141300,-8.613080,0,28,47,74,239
1,10009999,https://www.airbnb.com/rooms/10009999,Horto flat with small garden,Apartment,Entire home/apt,Real Bed,2,1125,flexible,4,1,2,0,1.0,"[Wifi, Wheelchair accessible, Kitchen, Free pa...",317,187.00,0,1,https://a0.muscache.com/im/pictures/5b408b9e-4...,0,1282196,https://www.airbnb.com/users/show/1282196,Ynaie,"Rio de Janeiro, State of Rio de Janeiro, Brazil",Not Specified,https://a0.muscache.com/im/pictures/9681e3cc-4...,https://a0.muscache.com/im/pictures/9681e3cc-4...,Jardim Botânico,93.127614,0,1,0,1,1,"[email, phone, facebook]","Rio de Janeiro, Rio de Janeiro, Brazil",Jardim Botânico,Jardim Botânico,Rio De Janeiro,Brazil,BR,Point,-22.966254,-43.230750,1,0,0,0,0
2,1001265,https://www.airbnb.com/rooms/1001265,Ocean View Waikiki Marina w/prkg,Condominium,Entire home/apt,Real Bed,3,365,strict_14_with_grace_period,2,1,1,96,1.0,"[TV, Cable TV, Wifi, Air conditioning, Pool, K...",115,100.00,0,1,https://a0.muscache.com/im/pictures/15037101/5...,84,5448114,https://www.airbnb.com/users/show/5448114,David,"Honolulu, Hawaii, United States",within an hour,https://a0.muscache.com/im/users/5448114/profi...,https://a0.muscache.com/im/users/5448114/profi...,Waikiki,98.000000,0,1,0,18,18,"[email, phone, reviews, kba]","Honolulu, HI, United States",Oʻahu,Primary Urban Center,Oahu,United States,US,Point,21.286340,-157.839190,1,16,46,76,343
3,10021707,https://www.airbnb.com/rooms/10021707,Private Room in Bushwick,Apartment,Private room,Real Bed,14,1125,flexible,1,1,1,1,1.5,"[Internet, Wifi, Air conditioning, Kitchen, Bu...",40,Not Specified,0,1,https://a0.muscache.com/im/pictures/72844c8c-f...,100,11275734,https://www.airbnb.com/users/show/11275734,Josh,"New York, New York, United States",Not Specified,https://a0.muscache.com/im/users/11275734/prof...,https://a0.muscache.com/im/users/11275734/prof...,Bushwick,93.127614,0,1,1,1,1,"[email, phone, reviews, kba]","Brooklyn, NY, United States",Brooklyn,Bushwick,New York,United States,US,Point,40.697910,-73.936150,1,0,0,0,0
4,10030955,https://www.airbnb.com/rooms/10030955,Apt Linda Vista Lagoa - Rio,Apartment,Private room,Real Bed,1,1125,flexible,2,1,1,0,2.0,"[TV, Cable TV, Internet, Wifi, Air conditionin...",701,250.00,0,1,https://a0.muscache.com/im/pictures/59c516bd-c...,0,51496939,https://www.airbnb.com/users/show/51496939,Livia,BR,Not Specified,https://a0.muscache.com/im/pictures/b7911710-9...,https://a0.muscache.com/im/pictures/b7911710-9...,Lagoa,93.127614,0,1,0,1,1,"[email, phone, jumio, government_id]","Rio de Janeiro, Rio de Janeiro, Brazil",Lagoa,Lagoa,Rio De Janeiro,Brazil,BR,Point,-22.971951,-43.205047,1,28,58,88,363


In [42]:
client.close()